In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
root = "https://www.ola.org/en/members/current"

In [93]:
driver = webdriver.Chrome()

In [15]:
driver.get(root)
        
# Wait for the div to be present
div_class = "member-list-row col-sm-12 col-md-12 col-lg-6 grid-view"
# div_class = "member-list-row col-sm-12 col-md-12 col-lg-6 grid-view"
wait = WebDriverWait(driver, 10)
div = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, div_class.replace(" ", "."))))

# Find all anchor tags within the div
links = [x.find_element(By.TAG_NAME, "a") for x in div]

# Extract href attributes
hrefs = [link.get_attribute('href') for link in links]

In [52]:
# get bills and motion hrefs
data = []
for href in hrefs:
    driver.get(f"{href}/activity")
    time.sleep(0.5)
    buttons = driver.find_elements(By.XPATH, "//div[@class='row']/div[@role='tablist']")
    buttons[0].click()
    buttons[1].click()

    bills_and_motions = driver.find_elements(By.XPATH, "//span[@class='field-content']/a")
    bills = {'bill_name': [], 'bill_href': []}
    motions = {'motion_name': [], 'motion_href': []}
    for elem in bills_and_motions:
        inner_text = elem.get_attribute('innerText')
        if inner_text.startswith("Bill"):
            href = elem.get_attribute('href')
            bills['bill_name'].append(inner_text)
            bills['bill_href'].append(href)
        elif inner_text.startswith("Motion"):
            href = elem.get_attribute('href')
            motions['motion_name'].append(inner_text)
            motions['motion_href'].append(href)
    data.append({'bills': bills, 'motions': motions})

In [74]:
## Get roles and names:
data2 = []
for href in hrefs:
    name = href.split('/')[-1].strip().replace('-', ' ')
    driver.get(href)
    time.sleep(0.5)

    # role_class = "view view-member view-id-member view-display-id-member_current_role_block js-view-dom-id-cb73e91df8d278d984d19d5169dbc58f006d1bf565d64f0bc08844f7167eb48a"
    # role = driver.find_element(By.XPATH, f"//div[@class='{role_class}']/div[@class='view-content']").get_attribute('innerText')
    try:
        role = driver.find_element(By.XPATH, "//div[@block='block-views-block-member-member-current-role-block']").get_attribute('innerText').strip('Current roles\n')
    except:
        role = ""

    try:
        location = driver.find_element(By.XPATH, "//p[@class='riding']").get_attribute('innerText')
    except:
        location = ""

    try:
        party = driver.find_element(By.XPATH, "//div[@class='colour-square']/..").get_attribute('innerText').strip()
    except:
        party = ""
    data2.append({'name': name, 'role': role, 'location': location, 'party': party})

In [80]:
# join
joined_data = []
for d1, d2 in zip(data, data2):
    d3 = d2 | d1
    joined_data.append(d3)

In [89]:
# get bill details
for x in joined_data:
    bill_hrefs = x['bills']['bill_href']
    explanatory_notes = []
    for href in bill_hrefs:
        driver.get(href)
        time.sleep(0.5)
        try:
            explanatory_note = driver.find_element(By.XPATH, "//div[@class='WordSection1']/p[@class='section']").get_attribute('innerText')
        except:
            explanatory_note = ""
        explanatory_notes.append(explanatory_note)
    x['bills']['explanatory_notes'] = explanatory_notes

In [115]:
# add motion details
for x in joined_data:
    motion_hrefs = x['motions']['motion_href']
    motion_names = x['motions']['motion_name']
    motion_detailed = []
    for motion_name, motion_href in zip(motion_names, motion_hrefs):
        driver.get(motion_href)
        time.sleep(0.5)
        h3_elements = driver.find_elements(By.TAG_NAME, "h3")
        motion_detail = ""
        for h3 in h3_elements:
            if "Motion 47" in h3.text.strip():
                try:
                    motion_detail = h3.find_element(By.XPATH, "../div").get_attribute('innerText')
                    past = True
                except:
                    motion_detail = ""
        motion_detailed.append(motion_detail)
    x['motions']['motion_details'] = motion_detailed

In [118]:
# save into individual dfs
mpps = []
all_bills = []
all_motions = []

In [120]:
for idx, x in enumerate(joined_data):
    mpps.append({'_id': idx, 'name': x['name'],
                 'role': x['role'],
                 'location': x['location'],
                 'party': x['party']
    })
    bills = x['bills']
    motions = x['motions']

    bills['mpp_id'] = [idx] * len(bills['bill_name'])
    motions['mpp_id'] = [idx] * len(motions['motion_name'])
    # flatten dict
    for idx in range(len(bills['mpp_id'])):
        all_bills.append({'mpp_id': bills['mpp_id'][idx], 
                          'bill_name': bills['bill_name'][idx], 
                          'explanatory_notes': bills['explanatory_notes'][idx]
                         })
    for idx in range(len(motions['mpp_id'])):
        all_motions.append({'mpp_id': motions['mpp_id'][idx], 
                          'motion_name': motions['motion_name'][idx], 
                          'motion_href': motions['motion_href'][idx]
                         })

In [131]:
pd.DataFrame(all_motions).to_csv('motions.csv', index=False)
pd.DataFrame(all_bills).to_csv('bills.csv', index=False)
pd.DataFrame(mpps).to_csv('mpps.csv', index=False)